In [1]:
import argparse
import numpy as np
import plotly
import plotly.figure_factory as ff
from skimage import measure
import sys,os
import trimesh
import open3d as o3d
from scipy.interpolate import Rbf
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud
from sklearn.neighbors import KDTree

### Define Naive, MLS, RBF Reconstruction

In [2]:
def knnsearch(Q, R, K):
    """
    KNNSEARCH   Linear k-nearest neighbor (KNN) search
    IDX = knnsearch(Q,R,K) searches the reference data set R (n x d array
    representing n points in a d-dimensional space) to find the k-nearest
    neighbors of each query point represented by eahc row of Q (m x d array).
    The results are stored in the (m x K) index array, IDX. 
    
    Rationality
    Linear KNN search is the simplest appraoch of KNN. The search is based on
    calculation of all distances. Therefore, it is normally believed only
    suitable for small data sets. However, other advanced approaches, such as
    kd-tree and delaunary become inefficient when d is large comparing to the
    number of data points.
    %
    See also, kdtree, nnsearch, delaunary, dsearch
    By Yi Cao at Cranfield University on 25 March 2008
    """

    N, M = Q.shape
    idx = np.zeros((N, K), dtype = int)
    D = np.zeros((N, K))
    fident = np.array_equal(Q, R)
    if K==1:
        for k in range(0, N):
            d = np.sum((R[:, :] - Q[k, :]) ** 2, axis=1)
            if fident:
                d[k] = float('inf')
            D[k] = np.min(d)
            idx[k] = np.argmin(d)
    else:
        for k in range(0, N):
            d = np.sum((R[:, :] - Q[k, :]) ** 2, axis=1)
            if fident:
                d[k] = float('inf')
            D[k, :] = np.sort(d)[:K]
            idx[k, :] = np.argsort(d)[:K]
    print("==>Nearest neighbour search completed!")
    return idx

In [3]:
def naiveReconstruction(Mesh_v,Mesh_nor):
    """
    surface reconstruction with an implicit function f(x,y,z) representing
    signed distance to the tangent plane of the surface point nearest to each 
    point (x,y,z)
    input: filename of a point cloud
    output: reconstructed mesh
    """
    points = Mesh_v
    normals = Mesh_nor


    # construct a 3D NxNxN grid containing the point cloud
    # each grid point stores the implicit function value
    # set N=16 for quick debugging, use *N=64* for reporting results
    N = 64
    max_dimensions = np.max(points,axis=0) # largest x, largest y, largest z coordinates among all surface points
    min_dimensions = np.min(points,axis=0) # smallest x, smallest y, smallest z coordinates among all surface points
    bounding_box_dimensions = max_dimensions - min_dimensions # compute the bounding box dimensions of the point cloud
    grid_spacing = max(bounding_box_dimensions)/(N-9) # each cell in the grid will have the same size
    X, Y, Z =np.meshgrid(list(np.arange(min_dimensions[0]-grid_spacing*4, max_dimensions[0]+grid_spacing*4, grid_spacing)),
                         list(np.arange(min_dimensions[1] - grid_spacing * 4, max_dimensions[1] + grid_spacing * 4,
                                    grid_spacing)),
                         list(np.arange(min_dimensions[2] - grid_spacing * 4, max_dimensions[2] + grid_spacing * 4,
                                    grid_spacing)))
    
    complete_mesh = np.array([X.reshape(-1), Y.reshape(-1), Z.reshape(-1)]).transpose()
    
    
    # idx stores the index to the nearest surface point for each grid point.
    # we use provided knnsearch function
    Q = complete_mesh # grid points
    R = points # surface points
    K = 1
    idx = knnsearch(Q, R, K)
    IF = np.zeros(shape=(Q.shape[0],1))
    # print((Q[idx[0]] - R[idx[0]])[0])
    
    for i in range(Q.shape[0]):
        normal = normals[idx[i]][0]
        p_minus_pj = Q[i] - R[idx[i]][0]
        IF[i] = normal[0] * p_minus_pj[0] + normal[1] * p_minus_pj[1] + normal[2] * p_minus_pj[2]

    IF = IF.reshape(X.shape)

    verts, simplices = measure.marching_cubes_classic(IF, 0)
    recon_mesh = trimesh.Trimesh(vertices = verts,faces = simplices)
    return recon_mesh

In [4]:
def mlsReconstruction(Mesh_v,Mesh_nor):
    """
    surface reconstruction with an implicit function f(x,y,z) representing
    MLS distance to the tangent plane of the input surface points 
    input: filename of a point cloud
    output: reconstructed mesh
    """
    
    points = Mesh_v
    normals = Mesh_nor


    # construct a 3D NxNxN grid containing the point cloud
    # each grid point stores the implicit function value
    # set N=16 for quick debugging, use *N=64* for reporting results
    N = 64
    max_dimensions = np.max(points,axis=0) # largest x, largest y, largest z coordinates among all surface points
    min_dimensions = np.min(points,axis=0) # smallest x, smallest y, smallest z coordinates among all surface points
    bounding_box_dimensions = max_dimensions - min_dimensions # compute the bounding box dimensions of the point cloud
    grid_spacing = max(bounding_box_dimensions)/(N-9) # each cell in the grid will have the same size
    X, Y, Z =np.meshgrid(list(np.arange(min_dimensions[0]-grid_spacing*4, max_dimensions[0]+grid_spacing*4, grid_spacing)),
                         list(np.arange(min_dimensions[1] - grid_spacing * 4, max_dimensions[1] + grid_spacing * 4,
                                    grid_spacing)),
                         list(np.arange(min_dimensions[2] - grid_spacing * 4, max_dimensions[2] + grid_spacing * 4,
                                    grid_spacing)))
    
    # idx stores the index to the nearest surface point for each grid point.
    # we use provided knnsearch function
    Q = np.array([X.reshape(-1), Y.reshape(-1), Z.reshape(-1)]).transpose()
    R = points
    K = 20
    idx = knnsearch(Q, R, K)
    IF = np.zeros(shape=(Q.shape[0],1))


    nearest_surface_point_to_self = knnsearch(R, R, 1)
    linalg = np.linalg.norm(R - R[nearest_surface_point_to_self][:,0], axis=1)
    distances_to_nearest_points = np.linalg.norm(R - R[nearest_surface_point_to_self][:,0], axis=1)
    beta = 2 * np.mean(distances_to_nearest_points)
    print(beta)

    for j in range(Q.shape[0]):
        sum_phi = 0
        for i in range(20):
            normal = normals[idx[j]][i]
            p_minus_pi = Q[j] - R[idx[j]][i]
            di_p = normal[0] * p_minus_pi[0] + normal[1] * p_minus_pi[1] + normal[2] * p_minus_pi[2]
            phi = np.exp(-1*(np.linalg.norm(p_minus_pi)**2)/beta**2)
            IF[j] += di_p*phi
            sum_phi += phi
        IF[j] = IF[j]/(sum_phi)

    IF = IF.reshape(X.shape)


    verts, simplices = measure.marching_cubes_classic(IF, 0)
    recon_mesh = trimesh.Trimesh(vertices = verts,faces = simplices)
    return recon_mesh

In [5]:
def rbfReconstruction(Mesh_v,Mesh_nor):
    """
    surface reconstruction with an implicit function f(x,y,z) computed
    through RBF interpolation of the input surface points and normals
    input: filename of a point cloud, parameter epsilon
    output: reconstructed mesh
    """

    points = Mesh_v
    normals = Mesh_nor

    epsilon = 1e-4
    
    # construct a 3D NxNxN grid containing the point cloud
    # each grid point stores the implicit function value
    # set N=16 for quick debugging, use *N=64* for reporting results
    N = 64
    max_dimensions = np.max(points,axis=0) # largest x, largest y, largest z coordinates among all surface points
    min_dimensions = np.min(points,axis=0) # smallest x, smallest y, smallest z coordinates among all surface points
    bounding_box_dimensions = max_dimensions - min_dimensions # compute the bounding box dimensions of the point cloud
    grid_spacing = max(bounding_box_dimensions)/(N-9) # each cell in the grid will have the same size
    X, Y, Z =np.meshgrid(list(np.arange(min_dimensions[0]-grid_spacing*4, max_dimensions[0]+grid_spacing*4, grid_spacing)),
                         list(np.arange(min_dimensions[1] - grid_spacing * 4, max_dimensions[1] + grid_spacing * 4,
                                    grid_spacing)),
                         list(np.arange(min_dimensions[2] - grid_spacing * 4, max_dimensions[2] + grid_spacing * 4,
                                    grid_spacing)))

    Q = np.array([X.reshape(-1), Y.reshape(-1), Z.reshape(-1)]).transpose()
    R = points
    IF = np.zeros(shape=(Q.shape[0], 1)) #this is your implicit function - fill it with correct values!
    fp = [0]*3*R.shape[0] #3N weights
    fp[R.shape[0]:R.shape[0]*2] = [epsilon]*R.shape[0]   #for p + eps*n
    fp[R.shape[0]*2:] = [-1*epsilon]*R.shape[0]   #for p - eps*n
    fp = np.array(fp)
    weights = np.zeros(3*R.shape[0])
    offsets_1 = [ p + epsilon * normals[i] for i, p in enumerate(R)]
    offsets_2 = [ p - epsilon * normals[i] for i, p in enumerate(R)]

    all_points = np.concatenate((R, offsets_1, offsets_2))   #3N points
    spline_phi = np.zeros((all_points.shape[0], all_points.shape[0]))
    

    for i in range(all_points.shape[0]):
        pi = all_points[i]
        for k in range(all_points.shape[0]):
            ck = all_points[k]
            r = np.linalg.norm(pi - ck) + 1e-8
            spline_phi[i][k] = (r**2) * np.log(r)

    weights = np.linalg.solve(spline_phi, fp)

    IF = np.zeros(shape=(Q.shape[0], 1))
    
    for i in range(Q.shape[0]):
        pi = Q[i]
        pi = pi.reshape((1,3))
        pi = np.repeat(pi, all_points.shape[0], axis=0)
        r_vector = np.linalg.norm(pi - all_points, axis=1) + 1e-8
        spline_phi_pi = (r_vector**2) * np.log(r_vector)
        IF[i] = np.dot(spline_phi_pi, weights)

    IF = IF.reshape(X.shape)

    verts, simplices = measure.marching_cubes_classic(IF,0)
    
    recon_mesh = trimesh.Trimesh(vertices = verts,faces = simplices)
    return recon_mesh

### The Plot function

In [6]:
def plotmesh(mesh):
    out = o3d.geometry.TriangleMesh()
    out.vertices=o3d.utility.Vector3dVector(mesh.vertices)
    out.triangles = o3d.utility.Vector3iVector(mesh.faces)
    out.compute_vertex_normals()
    o3d.visualization.draw_geometries([out])

In [7]:
def plotpoint(vertex):
    out = o3d.geometry.PointCloud()
    out.points=o3d.utility.Vector3dVector(vertex)
    out.paint_uniform_color((0.2, 0.3, 0.5))
    o3d.visualization.draw_geometries([out])

### Load mesh

In [8]:
# # mesh_1 = os.path.join('./meshes/camel.obj')
# mesh_1 = os.path.join('./meshes/bunny.obj')
# # mesh_1 = os.path.join('./meshes/wheel.obj')
# # mesh_1 = os.path.join('./meshes/dragon.obj')
# assert os.path.exists(mesh_1), 'cannot found:'+mesh_1 
# t_mesh_1 = trimesh.load(mesh_1) 
# t_mesh_1_v = t_mesh_1.vertices
# t_mesh_1_nor = t_mesh_1.vertex_normals


mesh = os.path.join('./meshes/camel.obj')
assert os.path.exists(mesh), 'cannot found:'+mesh
t_mesh = trimesh.load(mesh)
t_mesh_v = t_mesh.vertices
t_mesh_nor = t_mesh.vertex_normals

### Show the mesh

In [9]:
plotmesh(t_mesh)
# t_mesh_1.export(f'./results/wheel/v_cloud1.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [10]:
plotpoint(t_mesh_v)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### RBF Reconstruction

In [11]:
# verts1_rbf, simplices1_rbf = rbfReconstruction(t_mesh_1,1)
recon_rbf = rbfReconstruction(t_mesh_v,t_mesh_nor)

In [12]:
plotmesh(recon_rbf)
# recon_rbf.export('./results/bunny/bunny_rbf.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### Naive Reconstruction

In [13]:
recon_naive = naiveReconstruction(t_mesh_v,t_mesh_nor)

==>Nearest neighbour search completed!


In [14]:
plotmesh(recon_naive)
# recon_naive.export('./results/bunny/bunny_naive.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### MLS Reconstruction

In [15]:
recon_mls = mlsReconstruction(t_mesh_v,t_mesh_nor)

==>Nearest neighbour search completed!
==>Nearest neighbour search completed!
2.3751232270470295


In [16]:
plotmesh(recon_mls)
# recon_mls.export('./results/bunny/bunny_mls.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### Large Point Cloud Reconstruction

In [17]:
mesh_2 = os.path.join('./meshes/dragon.obj')
assert os.path.exists(mesh_2), 'cannot found:'+mesh_2
t_mesh_2 = trimesh.load(mesh_2)
t_mesh_2_v = t_mesh_2.vertices
t_mesh_2_nor = t_mesh_2.vertex_normals

In [18]:
recon2_naive = naiveReconstruction(t_mesh_2_v,t_mesh_2_nor)

==>Nearest neighbour search completed!


In [19]:
plotmesh(recon2_naive)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [20]:
recon2_mls = mlsReconstruction(t_mesh_2_v,t_mesh_2_nor)

==>Nearest neighbour search completed!
==>Nearest neighbour search completed!
0.0011449192967129


/var/folders/72/mr36hktn5yn4bxcxfmb80rn80000gn/T/ipykernel_88311/2943411806.py:51: RuntimeWarning: invalid value encountered in divide
  IF[j] = IF[j]/(sum_phi)
/Users/linyingxiang/opt/miniconda3/envs/geo_tutorials/lib/python3.9/site-packages/skimage/measure/_marching_cubes_classic.py:259: RuntimeWarning: invalid value encountered in divide
  grad_centroids = (grad_centroids /


In [21]:
plotmesh(recon2_mls)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### Noised Point Cloud Reconstruction

In [22]:
def add_Gaussian_noise(mesh,noise_level):
    vertices = np.array(mesh.vertices).copy()
    vertices_scale = np.max(vertices,axis=0)-np.min(vertices,axis=0)
    noise = noise_level * vertices_scale * np.random.randn(vertices.shape[0],vertices.shape[1])
    noisy_mesh_vertices = vertices + noise
    noisy_mesh = trimesh.Trimesh(vertices=noisy_mesh_vertices,faces=mesh.faces)
    return noisy_mesh

### Load the original mesh

In [23]:
mesh_ori = os.path.join('./meshes/dragon.obj')
assert os.path.exists(mesh_ori), 'cannot found:'+mesh_ori 
t_mesh_ori = trimesh.load(mesh_ori)
t_mesh_ori_v = t_mesh_ori.vertices
t_mesh_ori_nor = t_mesh_ori.vertex_normals

In [24]:
noise_mesh = add_Gaussian_noise(t_mesh_ori,0.001)
noise_mesh_v = noise_mesh.vertices
noise_mesh_nor = noise_mesh.vertex_normals
# plotpoint(noise_mesh_v)
plotmesh(noise_mesh)
# noise_mesh.export('./results/dragon/noise_dragon.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [118]:
noise_dragon_naive = naiveReconstruction(noise_dragon_v,noise_dragon_nor)

==>Nearest neighbour search completed!


In [155]:
plotfunction(noise_dragon_naive)
# noise_dragon_naive.export('./results/dragon/noise_dragon_naive.obj')

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [158]:
noise_dragon_mls = mlsReconstruction(noise_dragon_v,noise_dragon_nor)

==>Nearest neighbour search completed!
==>Nearest neighbour search completed!
0.001076919688734251


/var/folders/72/mr36hktn5yn4bxcxfmb80rn80000gn/T/ipykernel_67317/1606610570.py:53: RuntimeWarning: invalid value encountered in divide
  IF[j] = IF[j]/(sum_phi)
/Users/linyingxiang/opt/miniconda3/envs/geo_tutorials/lib/python3.9/site-packages/skimage/measure/_marching_cubes_classic.py:259: RuntimeWarning: invalid value encountered in divide
  grad_centroids = (grad_centroids /


In [137]:
plotfunction(noise_dragon_mls)
# noise_dragon_mls.export('./results/dragon/noise_dragon_mls.obj')

NameError: name 'noise_dragon_mls' is not defined